<h2>FT001 - Extracción ingresos y pasivos </h2>
<p> En este apartado se trata la base de datos FT001 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [12]:
# Se carga el FT001 completo desde el archivo .csv
FT001 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/FT001_1219_0521.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object})

In [15]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'codigoConcepto': 'codigo','claseConcepto': 'clase'}, inplace=True)

In [16]:
# Para hacer la suma de Valor por NIT EPS, Código del concepto, Mes y Año
FT001 = FT001.groupby(['nit_eps', 'codigo', 'ano', 'mes']).agg({'valor': 'sum'}).reset_index()

In [17]:
#Se transforman las variables a STR para mejorar su manipulación
FT001[['codigo']] = FT001[['codigo']].astype(float).astype(int).astype(str)

In [18]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT001['Lcod'] = FT001.codigo.str.len()

In [19]:
# Se limpia la clasificación por Tipo de deudor
FT001['tipo']= 'NA' # Se crea la columna con categoria UPC como base

# Se establecen las condiciones para establecer correctamente el UPC

FT001.loc[(FT001['codigo'] == "410201") |
       (FT001['codigo'] == "410202") |
       (FT001['codigo'] == "410203") |
       (FT001['codigo'] == "410207") |
       (FT001['codigo'] == "431101") |
       (FT001['codigo'] == "431102") |
       (FT001['codigo'] == "431106") |
       (FT001['codigo'] == "431122") |
       (FT001['codigo'] == "431120"),  
       'tipo'] = 'UPC' 

FT001.loc[(FT001['codigo'] == '431103') | # | Regimen contributivo
       (FT001['codigo'] == "431104") | #Regimen contributivo
       (FT001['codigo'] == "431107") | #Regimen ubsidiado publico
       (FT001['codigo'] == "41020801") | #410208-41020801
       (FT001['codigo'] == "410209"),  
       'tipo'] = 'CMCOP' 

FT001.loc[(FT001['codigo'] == '410227'),  
       'tipo'] = 'PMAX' 

FT001.loc[(FT001['codigo'] == '410230'),  
       'tipo'] = 'EnfH'

FT001.loc[(FT001['codigo'] == '1105') |
       (FT001['codigo'] == "110101") |
       (FT001['codigo'] == "110102") |
       (FT001['codigo'] == "110103") |
       (FT001['codigo'] == "110109"),  
       'tipo'] = 'Efectivo' 

In [20]:
# Para hacer la suma de Valor por NIT EPS, tipo, Mes y Año
FT001 = FT001.groupby(['nit_eps', 'tipo', 'mes', 'ano']).agg({'valor': 'sum'}).reset_index()

In [21]:
%%time
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT001 = FT001.set_index('mes').groupby(['nit_eps', 'tipo', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipo', 'ano'], 1)

Wall time: 375 ms


In [22]:
# Se desagrupa la información
FT001 = FT001.reset_index()

In [23]:
#Se transforman las variables a INT para mejorar su manipulación
FT001[['ano']] = FT001[['ano']].astype(float).astype(int)

In [24]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT001[
    (FT001['ano'] == 2019) & (FT001['mes'] == 1) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 2) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 3) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 4) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 5) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 6) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 7) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 8) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 9) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 10) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 11)
    ].index

In [25]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT001 = FT001.drop(df_filtered)

In [26]:
# Se establece el index adecuado para realizar la resta del valor. En este caso, se realiza la resta periodo a periodo (t+1 - t) el valor reportado en el FT001 para cada NIT EPS, tipo de deudor mes y año
FT001 = FT001.set_index(['nit_eps', 'tipo', 'mes', 'ano'])

In [27]:
# Se calcula la diferencia mensual (respetando las carcateristicas del index anteriomente establecido) para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT001_dif = FT001.diff().fillna(0)

In [28]:
# Se desagrupa la informacion
FT001_dif = FT001_dif.reset_index()
FT001 = FT001.reset_index()

In [29]:
# Se reemplazan los valores para los meses de enero
FT001_dif.loc[FT001_dif['mes'] == 1] = FT001

In [30]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001_dif.rename(columns={'valor': 'Dvalormes'}, inplace=True)

In [31]:
# Se encuentran las observaciones que cumplen las condiciones (se eliminan las observaciones de 2019)
df_filtered = FT001_dif[
    (FT001_dif['ano'] == 2019)
    ].index

In [32]:
# Se eliminan las observaciones que cumplen las condiciones anteriores (se eliminan las observaciones de 2019)
FT001_dif = FT001_dif.drop(df_filtered)

In [33]:
# Para convertir el tipo como columna para la base de datos con la diferencia de un periodo
FT001_dif = FT001_dif.pivot(index=['nit_eps', 'mes', 'ano'], columns='tipo', values='Dvalormes').fillna(0).reset_index()

In [34]:
# Para convertir el tipo como columna para la base de datos agregada
FT001 = FT001.pivot(index=['nit_eps', 'mes', 'ano'], columns='tipo', values='valor').fillna(0).reset_index()

In [35]:
# Se eliminan las columnas que no son de interés
FT001 = FT001.drop(['CMCOP', 'EnfH', 'NA', 'PMAX', 'UPC'], axis=1)

In [36]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001.rename(columns={'Efectivo': 'valorEfectivo'}, inplace=True)

In [37]:
# Se unen las dos bases de datos para obtener la información en conjunto de la diferencia y el valor de Efectivo
FT001_dif = pd.merge(FT001_dif, FT001, how='left', left_on=['nit_eps', 'ano', 'mes'], right_on=['nit_eps', 'ano', 'mes'])

In [38]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001_dif.rename(columns={'Efectivo': 'DEfectivo', 'CMCOP': 'DCMCOPmes', 'PMAX': 'DPMAXmes', 'UPC': 'DUPCmes'}, inplace=True)

In [39]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT001_dif.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/ingresosyefectivo_2020_2021_V2.xlsx', index = False)
FT001_dif.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/ingresosyefectivo_2020_2021_V2.csv', index = False, encoding='utf-8-sig')

In [40]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT001_dif, FT001, df_filtered, fecha_ideal]